In [1]:
import pandas as pd
import sqlite3
import twint
import subprocess

In [27]:
!twint -u narendramodi --following -o nm_follow.csv --csv

SandeepParekh
jairbolsonaro
Kuldeep78328755
Temple_Phantom
swetasamadhiya
thakkar_sameet
loksabhaspeaker
fskulaste
Dchautala
kataria4ambala
nityanandraibjp
Phogat_Vinesh
poojadhanda0007
imranirampal
IamBhavaniDevi
DipaKarmakar
SakshiMalik
amit_jurno
nadeemNBT
AmodRaiLive
NayantaraRai
pranabsamanta
iBackModi
shreyashsingh07
p_pavanraj
raiprakashji
Aish17aer
vin8737
pundreekmishra
im_S_pandey
YashvikramTrip
iamberagarwal
SanthaVb
kaulsanjay
Arvindgmt
adikulk
Reetesh777
ibijendrasharma
amitabhnews18
WordsSlay
drashwathcn
krishna_pravin
iSurjeetSharma
RishabhMtweets
deepscribble
nidhitripathi92
arvindneela
sumati_mehrishi
kapskom
utkarsh_aanand
gunjakapoor
nkgrock
IamNaveenKapoor
neelkanthmishra
manishBJPUP
poojakapil4
shalabhmani
abhilashBJPmp
TheSatishDua
raghavohri0
PrinceArihan
JournoPranay
KiniColonel
ajitbhinder
imAditiofficial
DrGeetaBhatt
PriyankaChhawri
GauriSheoran
TahiraAzmet
HemlataTanwar6
shikhadeepsri
Meenaparmar1977
pn906
mohit_db10
ARCHANA2371226
priyanka4nation
RanvirShore

RShivshankar
navikakumar
MajDPSingh
FrustIndian
authoramish
vkendra
visrane
CitiznMukherjee
M_Raj03
mandhana_smriti
ImHarmanpreet
vedakmurthy08
ImSushVerma
RomaSahni
rashtrapatibhvn
erna_solberg
PaoloGentiloni
netanyahu
DurgaMenon
DileepJaisalmer
Avantika_23
priyaakulkarni2
MODIfiedMayank
achandwar
MihirMisra_
rakeshrjnair
harshgupta1961
isagarsinghbjp
MahiHerambha
Rajsharma4CG
SHAHSUDH
sajithpremadasa
winningalways
BJP_RSS
01Karn
Seems3r
neelang_dave
harish_purohit
I_YogiVerma
EmmanuelMacron
dineshjoshi70
desh_bhkt
ThackerNikhil
DPrasanthNair
Anmologist
theRohitBansal
ShamikaRavi
iSengarAjayy
RishiMishra_
Indianit07
DrSinghAnuj
murli_dhar
ankurjain200607
dhruvbhim
AmitSharma20482
trpresidency
RTErdogan
iamnagarjuna
praveen_bajpai
Khandelw13Sagar
rohit_chahal
Shreeharib1
manisha_111
AnastasiadesCY
HritikSingh_
WeDeeptiSumit
JontyRhodes8
Ramlal
AnoopKaippalli
VRamanuj
AYBPS
sidharth_naik
KANABARDr
NayakDeep
neelshah31790
SunilYadavBJP
Navrang
ivishalthacker
radhichappi
mishra_shani
sure

sarbanandsonwal
JPNadda
irvpaswan
hannover_messe
NSC44
MOFPI_GOI
minmsme
bhaktisharma4
PtDhirOfficial
Chopsyturvey
nishahnt
FinMinIndia
nasscom
Leander
rohanbopanna
rahulroushan
UdayMahurkar
GoI_MeitY
Arpithams
saureen
prasarbharati
leehsienloong
satyanadella
AndrewRobbAO
dhume
mpgaruda
RajanTiwari_
AdvaitaKala
Elysee
LaurentFabius
mrlanceprice
MaithripalaS
CKforCleanIndia
MikeBloomberg
imjadeja
RayuduAmbati
ImRaina
ajinkyarahane88
ImRo45
SDhawan25
imVkohli
ashwinravi99
BhuviOfficial
y_umesh
JohnKerry
IsraeliPM
KremlinRussia_E
MohamedNasheed
MJameelAhmed
MedvedevRussiaE
FijiPM
fhollande
mlkhattar
Baird
GovChristie
MarsOrbiter
isro
RtHon_JohnKey
HonTonyAbbott
dilmabr
POI13
tsheringtobgay
MEAIndia
nitin_gadkari
rajinikanth
ptshrikant
ShainaNC
anugyaan
ananya_goenka
sadhuvaad
LibraRulz
balamy
rasita
Tan_Tripathi
Wisey_woman
monicanitin
Ankitaaa_
Chhabiy
1SH4N
SriSutaa
Deveshvari
IndianMaverick
mohitbajpai
amritabhinder
smarket
indianalltheway
AlSanghi
MsWeera
RnRSquall
iAsura_
tweetingsou

SushilModi
v_shrivsatish
VasundharaBJP
alok_bhatt
DhrumilBJP
PiyushGoyal
CaptAbhimanyu
sirudrani
vibhask1
MLASangitaPatil
ericschmidt
thevaibhavag
tathagata2
AP_BJP_IT_Cell
BJPKarSMITCell
BJP4Delhi
USIBC
sarkar_swati
DrChandanMitra
HugoSwire
DrShobha
KittuKrithika
JamesBevanEA
AnanthKumar_BJP
David_Cameron
AbeShinzo
RNTata2000
TOIAhmedabad
oneindiaHindi
oneindiagujarat
PonnaarrBJP
PMuralidharRao
Hiteshchaudhari
Rajan_Tewari
DushyantDholpur
moderateright
ParagDr
RohitBJP
RSSorg
veer
avinash_b01
harshaperla
kushanmitra
rvaidya2000
smna17
UtsavMitra
mepratap
sunandavashisht
JapanPathak
gujratsamachar
JPN_PMO
indianyogi
amitvkaushik
manishailesh
manojladwa
rammadhavbjp
5Forty3
Ashutosh1671
Pankajbhaidesai
paritosh2608
ianuragthakur
sanjay_kaul
TimesNow
sandeshnews
Divya_Bhaskar
jainil2988
cppatel35
ihardikshah
jaysinhbjp1
TravelQuotient
nanditathhakur
pawankaushal
MadhaviBhuta
VijayGoelBJP
KalamCenter
arrahman
NitiCentral
vijayrupanibjp
amitabhk87
NarayanbhaiPate
ajitsinh_
Sharnarthee
Vira

In [2]:
!ls -l

total 716
-rw-r--r-- 1 rakeshbhat9 rakeshbhat9 509222 May 25 12:06 cb_may_test.csv
-rw-r--r-- 1 rakeshbhat9 rakeshbhat9  40742 May 25 14:26 nm_follow-Copy1.csv
-rw-r--r-- 1 rakeshbhat9 rakeshbhat9  40742 May 25 14:11 nm_follow.csv
-rw-r--r-- 1 rakeshbhat9 rakeshbhat9  67529 May 25 14:39 nm_follow.ipynb
-rw-r--r-- 1 rakeshbhat9 rakeshbhat9  66281 May 25 12:32 twint_test.ipynb


In [7]:
d = pd.read_csv('nm_follow.csv')

In [8]:
d.shape

(2364, 1)

In [9]:
d.head()

,username
0,sandeepparekh
1,jairbolsonaro
2,kuldeep78328755
3,temple_phantom
4,swetasamadhiya


In [ ]:
subprocess.check_call(
    [twint -u  --since '2020-05-01 00:00:00' -s muslim -limit 10]
    shell=True)

In [15]:
for user in d['username'].iloc[:2]:
    subprocess.check_call(["twint" ,"-u" ,user ,"--since",'2020-05-01 00:00:00',"-s" ,"muslim" ,"-limit" ,"10"],shell=True)

In [17]:
user

'jairbolsonaro'

In [3]:
!twint -u jairbolsonaro --since '2020-05-30 00:00:00' 

1266806815175716864 2020-05-30 20:00:55 BST <jairbolsonaro> Hoje, entre diversos compromissos e conversas, acompanhei, junto do @JusticaGovBR e @tarcisiogdf , uma das ações da @PRFBrasil , nominada de Operação Tamoios, na BR 060, entre o DF e Goiânia. Durante a Operação, que começou no dia 28, mais de 7,7TON de maconha foram apreendidas. pic.twitter.com/sa6LHIDsLe
1266796938911010820 2020-05-30 19:21:40 BST <jairbolsonaro> 🇧🇷🤝
1266781861721845760 2020-05-30 18:21:46 BST <jairbolsonaro> - Mais um vídeo de Alexandre de Moraes e as liberdades:  . Link no YouTube:  https://youtu.be/IO_WiZSjarQ  pic.twitter.com/WD2kpLf1ua
1266748034337198082 2020-05-30 16:07:21 BST <jairbolsonaro> - Como muitos profissionais, Nobel de Química e as consequências das medidas de isolamento. pic.twitter.com/u50bCOdpj0
1266518583439392768 2020-05-30 00:55:35 BST <jairbolsonaro> - Todos sabem que durante o mandato p/ o qual fui eleito, que vai até 2022, estão previstas apenas 2 vagas p/ o Supremo Tribunal Federal

In [11]:
# %%bash
# sed 1d nm_follow.csv | head -n 5 | cut -d ',' -f2 |while read line 
# do
#    twint -u $line --since '2020-05-01 00:00:00' -s muslim -limit 10
# done

In [22]:
!twint --help

usage: python3 twint [options]

TWINT - An Advanced Twitter Scraping Tool.

optional arguments:
  -h, --help            show this help message and exit
  -u USERNAME, --username USERNAME
                        User's Tweets you want to scrape.
  -s SEARCH, --search SEARCH
                        Search for Tweets containing this word or phrase.
  -g GEO, --geo GEO     Search for geocoded Tweets.
  --near NEAR           Near a specified city.
  --location            Show user's location (Experimental).
  -l LANG, --lang LANG  Search for Tweets in a specific language.
  -o OUTPUT, --output OUTPUT
                        Save output to a file.
  -es ELASTICSEARCH, --elasticsearch ELASTICSEARCH
                        Index to Elasticsearch.
  --year YEAR           Filter Tweets before specified year.
  --since DATE          Filter Tweets sent since date (Example: "2017-12-27
                        20:30:15" or 2017-12-27).
  --until DATE          Filter Tweets sent until date (Example: 

In [20]:
# d = pd.read_csv('followers_tweet.csv',error_bad_lines=False)

In [14]:
with open('followers_tweet.csv','r') as f:
    print(len(f.readlines()))

357


In [1]:
!twint -u swetasamadhiya --since "2020-05-01 00:00:00" \
        --min-likes 100 \
        -fr \
        --limit 1 \
        -l en
        # --translate 

1266363606125301762 2020-05-29 14:39:46 BST <swetasamadhiya> Done 🤘 😸😸 pic.twitter.com/jc7aiUwOSR
1266110579263115264 2020-05-28 21:54:19 BST <swetasamadhiya> Don't think about us now our country's head is able to handle all the situation .  Congress era was vanished in 2016 ,  #India
1260293450370633733 2020-05-12 20:39:08 BST <swetasamadhiya> Party without leader 👏😂 https://twitter.com/SalmanNizami_/status/1260247585731702786 …
